In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b0')

In [ ]:
model_name = 'efficientnet-b0'
image_size = EfficientNet.get_image_size(model_name)

In [ ]:
image_size

In [ ]:
model = EfficientNet.from_pretrained(model_name)

In [ ]:
model

In [ ]:
import pandas as pd


In [ ]:
np.random.seed(42)


In [ ]:
src = (ImageList.from_folder(path='train-snakes-cropped').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
res = []
res.append(rotate(degrees=(-90,90), p=1))

In [ ]:
res

In [ ]:
tfms=(res,[])

In [ ]:
tfms[0]

In [ ]:
bs=96

In [ ]:
data = (src.transform(tfms, size=image_size, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=bs).normalize(imagenet_stats))

In [ ]:
data.show_batch(3,figsize=(9,9))

In [ ]:
list(children(model))[-2:]


In [ ]:
model

In [ ]:
model.add_module('_fc',nn.Linear(1280, data.c))


In [ ]:
loss_func = LabelSmoothingCrossEntropy()

In [ ]:
RMSprop = partial(torch.optim.RMSprop,eps=1e-6,centered=True)


In [ ]:
from torch.optim import Optimizer

In [ ]:
learn = Learner(data, model, loss_func=loss_func, opt_func=RMSprop, metrics=[accuracy,FBeta(beta=1,average='macro')],bn_wd=False)

In [ ]:
learn.model

In [ ]:
learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:2]],
             [learn.model._blocks[2:],learn.model._conv_head], 
             [learn.model._bn1,learn.model._fc]])

In [ ]:
learn.summary()

In [ ]:
learn.lr_find()


In [ ]:
learn.recorder.plot()

In [ ]:
lr=1e-3

In [ ]:
learn.fit(2,slice(lr/100,lr))

In [ ]:
learn.summary()

In [ ]:
lr=lr/100

In [ ]:
learn.fit(2,slice(lr/100,lr))

In [ ]:
learn.save('b0-fl-round-1')

In [ ]:
learn.fit_one_cycle(20,max_lr=slice(lr/1000,lr/100))

In [ ]:
learn.save('b0-f1-round-2')